# 🎯 Módulo 11: Avaliação de Agentes - Métricas, Testes e Performance

## Curso Completo Agno 2.0 Framework



**Olá, pessoal! Pedro Guth aqui! 👋**

Tá, mas o que adianta criar agentes incríveis se não sabemos se eles estão funcionando bem? É como ter um carro esportivo mas nunca olhar o painel! 🏎️

Neste módulo vamos aprender a **medir, avaliar e otimizar** nossos agentes. Bora descobrir se nossos "robôs" estão trabalhando direitinho!

### 🎯 O que vamos aprender:
- Como medir a performance dos agentes
- Métricas importantes para avaliar
- Testes automatizados para agentes
- Monitoramento em tempo real
- Otimização baseada em dados

## 🚀 Setup Inicial - Preparando o Ambiente

Vamos começar instalando tudo que precisamos para avaliar nossos agentes como verdadeiros profissionais!

In [ ]:
# Instalação dos pacotes necessários
!pip install -q agno google-generativeai matplotlib seaborn pandas numpy plotly

In [ ]:
# Imports principais
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
from typing import List, Dict, Any

# Configuração visual
plt.style.use('default')
sns.set_palette("husl")

print("🎯 Ambiente preparado! Bora avaliar alguns agentes!")

In [ ]:
# Configuração da API do Google (OBRIGATÓRIO - Cole sua API Key aqui!)
import google.generativeai as genai
from getpass import getpass

# Cole sua API Key do Google AI Studio aqui
api_key = getpass("🔑 Cole sua API Key do Google AI Studio: ")
genai.configure(api_key=api_key)

print("✅ API configurada! Agora vamos aos agentes!")

## 🤔 Por Que Avaliar Agentes?

Imagina que você tem uma padaria e contrata funcionários. Como saber se eles estão fazendo um bom trabalho? Você mede:

- **Qualidade**: O pão está gostoso? ⭐
- **Velocidade**: Quanto tempo demora pra fazer? ⚡
- **Consistência**: Sempre igual ou varia muito? 📊
- **Custo**: Vale o que custa? 💰

Com agentes é **exatamente igual**! Precisamos medir se nossos "funcionários digitais" estão entregando valor.

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_02.png)

### 📊 Principais Métricas de Agentes:

#### 1. **Métricas de Qualidade**
- Precisão das respostas
- Relevância do conteúdo
- Coerência lógica

#### 2. **Métricas de Performance**
- Tempo de resposta
- Tokens utilizados
- Custo por interação

#### 3. **Métricas de Comportamento**
- Taxa de sucesso em tarefas
- Uso adequado de ferramentas
- Seguimento de instruções

## 🛠️ Criando um Agente para Testar

Vamos criar um agente simples que vamos usar como "cobaia" nos nossos testes. Vai ser divertido! 😄

In [ ]:
# Importando o Agno
from agno.agent import Agent
from agno.models.google import Gemini

# Criando nosso agente "cobaia"
agente_teste = Agent(
    name="Assistente Matemático",
    model=Gemini(id="gemini-2.0-flash-exp"),
    description="Você é um assistente especializado em matemática que resolve problemas de forma clara e didática.",
    instructions=[
        "Sempre explique o passo a passo das soluções",
        "Use exemplos práticos quando possível",
        "Se não souber algo, seja honesto"
    ],
    markdown=True
)

print("🤖 Agente criado! Agora vamos testá-lo...")

In [ ]:
# Teste rápido do agente
print("🧪 Fazendo um teste rápido...")
resposta = agente_teste.run("Quanto é 2 + 2?")
print(f"Resposta: {resposta.content}")
print(f"Tempo: {resposta.metrics.duration:.2f} segundos")
print(f"Tokens: {resposta.metrics.input_tokens} entrada + {resposta.metrics.output_tokens} saída")

## 📊 Coletando Métricas Básicas

Agora vamos criar uma função para coletar métricas detalhadas de cada interação. É como ter um "contador" para tudo que o agente faz!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_03.png)

In [ ]:
# Função para coletar métricas detalhadas
def coletar_metricas(agente, pergunta, resposta_esperada=None):
    """Coleta métricas detalhadas de uma interação com o agente"""
    
    # Marca o tempo de início
    inicio = time.time()
    
    # Executa o agente
    try:
        resposta = agente.run(pergunta)
        sucesso = True
        erro = None
    except Exception as e:
        resposta = None
        sucesso = False
        erro = str(e)
    
    # Calcula tempo total
    tempo_total = time.time() - inicio
    
    # Monta o dicionário de métricas
    metricas = {
        'timestamp': datetime.now(),
        'pergunta': pergunta,
        'resposta': resposta.content if resposta else None,
        'sucesso': sucesso,
        'erro': erro,
        'tempo_total': tempo_total,
        'tokens_entrada': resposta.metrics.input_tokens if resposta else 0,
        'tokens_saida': resposta.metrics.output_tokens if resposta else 0,
        'tokens_total': (resposta.metrics.input_tokens + resposta.metrics.output_tokens) if resposta else 0,
        'duracao_modelo': resposta.metrics.duration if resposta else 0,
    }
    
    # Se temos resposta esperada, calcula similaridade básica
    if resposta_esperada and resposta:
        # Similaridade simples baseada em palavras comuns
        palavras_resposta = set(resposta.content.lower().split())
        palavras_esperada = set(resposta_esperada.lower().split())
        
        if len(palavras_esperada) > 0:
            similaridade = len(palavras_resposta.intersection(palavras_esperada)) / len(palavras_esperada)
        else:
            similaridade = 0
            
        metricas['similaridade_esperada'] = similaridade
    
    return metricas

print("✅ Função de métricas criada! Vamos usar ela nos testes.")

## 🧪 Executando Bateria de Testes

Vamos criar uma bateria de testes para nosso agente. É como fazer vários "exames" para ver como ele se sai em diferentes situações!

**Dica!** Sempre teste com casos variados - fáceis, difíceis e até pegadinhas! 😉

In [ ]:
# Bateria de testes para o agente matemático
casos_teste = [
    {
        'pergunta': 'Quanto é 5 + 3?',
        'resposta_esperada': '8 cinco mais três igual oito',
        'categoria': 'Básico',
        'dificuldade': 1
    },
    {
        'pergunta': 'Resolva a equação: 2x + 4 = 10',
        'resposta_esperada': 'x = 3 três solução equação',
        'categoria': 'Intermediário',
        'dificuldade': 3
    },
    {
        'pergunta': 'Qual a derivada de x² + 3x?',
        'resposta_esperada': '2x + 3 derivada cálculo',
        'categoria': 'Avançado',
        'dificuldade': 5
    },
    {
        'pergunta': 'Explique o teorema de Pitágoras',
        'resposta_esperada': 'hipotenusa catetos quadrado triângulo retângulo',
        'categoria': 'Conceitual',
        'dificuldade': 2
    },
    {
        'pergunta': 'Como fazer uma pizza?',
        'resposta_esperada': 'não sei matemática especialidade',
        'categoria': 'Fora do escopo',
        'dificuldade': 1
    }
]

print(f"📋 Preparados {len(casos_teste)} casos de teste!")
print("Categorias:", list(set([caso['categoria'] for caso in casos_teste])))

In [ ]:
# Executando todos os testes
print("🚀 Iniciando bateria de testes...")
print("=" * 50)

resultados = []

for i, caso in enumerate(casos_teste, 1):
    print(f"\n🧪 Teste {i}/{len(casos_teste)}: {caso['categoria']}")
    print(f"Pergunta: {caso['pergunta']}")
    
    # Coleta métricas
    metricas = coletar_metricas(
        agente_teste, 
        caso['pergunta'], 
        caso['resposta_esperada']
    )
    
    # Adiciona informações do caso de teste
    metricas['categoria'] = caso['categoria']
    metricas['dificuldade'] = caso['dificuldade']
    
    resultados.append(metricas)
    
    # Mostra resultados básicos
    if metricas['sucesso']:
        print(f"✅ Sucesso! Tempo: {metricas['tempo_total']:.2f}s | Tokens: {metricas['tokens_total']}")
        if 'similaridade_esperada' in metricas:
            print(f"📊 Similaridade: {metricas['similaridade_esperada']:.2f}")
    else:
        print(f"❌ Erro: {metricas['erro']}")

print("\n🎉 Bateria de testes concluída!")

## 📈 Analisando os Resultados

Agora vem a parte mais legal - analisar os dados! Vamos transformar números em insights úteis.

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_04.png)

In [ ]:
# Convertendo resultados para DataFrame
df_resultados = pd.DataFrame(resultados)

# Estatísticas básicas
print("📊 RESUMO GERAL DOS TESTES")
print("=" * 40)

# Taxa de sucesso
taxa_sucesso = df_resultados['sucesso'].mean() * 100
print(f"✅ Taxa de Sucesso: {taxa_sucesso:.1f}%")

# Métricas de tempo
tempo_medio = df_resultados[df_resultados['sucesso']]['tempo_total'].mean()
tempo_max = df_resultados[df_resultados['sucesso']]['tempo_total'].max()
tempo_min = df_resultados[df_resultados['sucesso']]['tempo_total'].min()

print(f"⏱️  Tempo Médio: {tempo_medio:.2f}s")
print(f"⏱️  Tempo Máximo: {tempo_max:.2f}s")
print(f"⏱️  Tempo Mínimo: {tempo_min:.2f}s")

# Métricas de tokens
tokens_medio = df_resultados[df_resultados['sucesso']]['tokens_total'].mean()
tokens_max = df_resultados[df_resultados['sucesso']]['tokens_total'].max()

print(f"🔢 Tokens Médio: {tokens_medio:.0f}")
print(f"🔢 Tokens Máximo: {tokens_max:.0f}")

# Similaridade média
df_com_similaridade = df_resultados[df_resultados['similaridade_esperada'].notna()]
if len(df_com_similaridade) > 0:
    similaridade_media = df_com_similaridade['similaridade_esperada'].mean()
    print(f"🎯 Similaridade Média: {similaridade_media:.2f}")

In [ ]:
# Análise por categoria
print("\n📋 ANÁLISE POR CATEGORIA")
print("=" * 40)

analise_categoria = df_resultados.groupby('categoria').agg({
    'sucesso': 'mean',
    'tempo_total': 'mean',
    'tokens_total': 'mean',
    'similaridade_esperada': 'mean'
}).round(2)

# Renomeia colunas para ficar mais bonito
analise_categoria.columns = ['Taxa Sucesso', 'Tempo Médio', 'Tokens Médios', 'Similaridade']
analise_categoria['Taxa Sucesso'] = (analise_categoria['Taxa Sucesso'] * 100).round(1)

print(analise_categoria)

# Identifica melhor e pior categoria
melhor_categoria = analise_categoria['Taxa Sucesso'].idxmax()
pior_categoria = analise_categoria['Taxa Sucesso'].idxmin()

print(f"\n🏆 Melhor Performance: {melhor_categoria}")
print(f"⚠️  Precisa Melhorar: {pior_categoria}")

## 📊 Visualizações das Métricas

Uma imagem vale mais que mil palavras! Vamos criar gráficos lindos para entender melhor a performance do nosso agente.

**Dica!** Gráficos são fundamentais para comunicar resultados para stakeholders! 📈

In [ ]:
# Configurando o layout dos gráficos
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 Dashboard de Performance do Agente', fontsize=16, fontweight='bold')

# Gráfico 1: Taxa de sucesso por categoria
categorias = df_resultados['categoria'].value_counts()
sucessos = df_resultados[df_resultados['sucesso']]['categoria'].value_counts()
taxa_por_categoria = (sucessos / categorias * 100).fillna(0)

axes[0,0].bar(taxa_por_categoria.index, taxa_por_categoria.values, color='skyblue', alpha=0.7)
axes[0,0].set_title('Taxa de Sucesso por Categoria')
axes[0,0].set_ylabel('Taxa de Sucesso (%)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# Gráfico 2: Distribuição do tempo de resposta
tempos_sucesso = df_resultados[df_resultados['sucesso']]['tempo_total']
axes[0,1].hist(tempos_sucesso, bins=5, color='lightgreen', alpha=0.7, edgecolor='black')
axes[0,1].set_title('Distribuição do Tempo de Resposta')
axes[0,1].set_xlabel('Tempo (segundos)')
axes[0,1].set_ylabel('Frequência')
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Tokens por dificuldade
df_sucesso = df_resultados[df_resultados['sucesso']]
dificuldades = sorted(df_sucesso['dificuldade'].unique())
tokens_por_dificuldade = [df_sucesso[df_sucesso['dificuldade']==d]['tokens_total'].mean() for d in dificuldades]

axes[1,0].plot(dificuldades, tokens_por_dificuldade, marker='o', linewidth=2, markersize=8, color='orange')
axes[1,0].set_title('Tokens vs Dificuldade')
axes[1,0].set_xlabel('Nível de Dificuldade')
axes[1,0].set_ylabel('Tokens Médios')
axes[1,0].grid(True, alpha=0.3)

# Gráfico 4: Similaridade por categoria
df_com_sim = df_resultados[df_resultados['similaridade_esperada'].notna()]
if len(df_com_sim) > 0:
    sim_por_cat = df_com_sim.groupby('categoria')['similaridade_esperada'].mean()
    axes[1,1].bar(sim_por_cat.index, sim_por_cat.values, color='coral', alpha=0.7)
    axes[1,1].set_title('Similaridade Média por Categoria')
    axes[1,1].set_ylabel('Similaridade')
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].grid(True, alpha=0.3)
else:
    axes[1,1].text(0.5, 0.5, 'Sem dados de\nsimilaridade', ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Similaridade por Categoria')

plt.tight_layout()
plt.show()

print("📊 Dashboard gerado! Olha só que lindo ficou! 🎨")

## ⚡ Monitoramento em Tempo Real

Agora vamos simular um monitoramento em tempo real - como se nosso agente estivesse em produção recebendo requests! 

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_05.png)

In [ ]:
# Simulador de monitoramento em tempo real
import random
from collections import deque

class MonitorAgente:
    def __init__(self, max_historico=50):
        self.historico_tempo = deque(maxlen=max_historico)
        self.historico_tokens = deque(maxlen=max_historico)
        self.historico_sucesso = deque(maxlen=max_historico)
        self.total_requests = 0
        self.total_erros = 0
        
    def registrar_interacao(self, metricas):
        """Registra uma nova interação no monitor"""
        self.total_requests += 1
        
        if metricas['sucesso']:
            self.historico_tempo.append(metricas['tempo_total'])
            self.historico_tokens.append(metricas['tokens_total'])
            self.historico_sucesso.append(1)
        else:
            self.total_erros += 1
            self.historico_sucesso.append(0)
            
    def obter_metricas_atuais(self):
        """Obtém métricas atualizadas"""
        if not self.historico_tempo:
            return None
            
        return {
            'tempo_medio_recente': sum(list(self.historico_tempo)[-10:]) / min(10, len(self.historico_tempo)),
            'tokens_medio_recente': sum(list(self.historico_tokens)[-10:]) / min(10, len(self.historico_tokens)),
            'taxa_sucesso_recente': sum(list(self.historico_sucesso)[-10:]) / min(10, len(self.historico_sucesso)) * 100,
            'total_requests': self.total_requests,
            'total_erros': self.total_erros,
            'taxa_erro_geral': self.total_erros / self.total_requests * 100 if self.total_requests > 0 else 0
        }
    
    def detectar_anomalias(self):
        """Detecta possíveis problemas"""
        alertas = []
        metricas = self.obter_metricas_atuais()
        
        if not metricas:
            return alertas
            
        # Alerta de tempo alto
        if metricas['tempo_medio_recente'] > 10:
            alertas.append(f"⚠️ Tempo de resposta alto: {metricas['tempo_medio_recente']:.2f}s")
            
        # Alerta de taxa de erro
        if metricas['taxa_erro_geral'] > 20:
            alertas.append(f"🚨 Taxa de erro alta: {metricas['taxa_erro_geral']:.1f}%")
            
        # Alerta de consumo de tokens
        if metricas['tokens_medio_recente'] > 1000:
            alertas.append(f"💰 Alto consumo de tokens: {metricas['tokens_medio_recente']:.0f}")
            
        return alertas

# Criando monitor
monitor = MonitorAgente()
print("📡 Monitor criado! Vamos simular algumas interações...")

In [ ]:
# Simulando interações em tempo real
perguntas_simulacao = [
    "Quanto é 10 + 5?",
    "Resolva: 3x - 6 = 9",
    "Qual a área de um círculo com raio 5?",
    "Explique a regra de três",
    "Como calcular porcentagem?"
]

print("🚀 Simulando tráfego em tempo real...")
print("=" * 50)

for i in range(15):  # Simula 15 interações
    # Escolhe uma pergunta aleatória
    pergunta = random.choice(perguntas_simulacao)
    
    print(f"\n📝 Request {i+1}: {pergunta[:30]}...")
    
    # Coleta métricas
    metricas = coletar_metricas(agente_teste, pergunta)
    
    # Registra no monitor
    monitor.registrar_interacao(metricas)
    
    # Obtém métricas atuais
    metricas_atuais = monitor.obter_metricas_atuais()
    
    if metricas_atuais:
        print(f"⏱️ Tempo médio: {metricas_atuais['tempo_medio_recente']:.2f}s")
        print(f"🔢 Tokens médio: {metricas_atuais['tokens_medio_recente']:.0f}")
        print(f"✅ Taxa sucesso: {metricas_atuais['taxa_sucesso_recente']:.1f}%")
        
        # Verifica alertas
        alertas = monitor.detectar_anomalias()
        for alerta in alertas:
            print(alerta)
    
    # Simula intervalo entre requests
    time.sleep(0.5)

print("\n🎯 Simulação concluída! Vamos ver o resumo final...")

## 🎯 Testes A/B entre Agentes

E se quisermos comparar diferentes versões do nosso agente? Vamos fazer um teste A/B! É como testar duas receitas de bolo para ver qual fica mais gostosa! 🍰

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_06.png)

In [ ]:
# Criando versão alternativa do agente (Versão B)
agente_b = Agent(
    name="Assistente Matemático v2",
    model=Gemini(id="gemini-2.0-flash-exp"),
    description="Você é um professor de matemática experiente que ensina de forma prática e divertida.",
    instructions=[
        "Use exemplos do dia a dia em suas explicações",
        "Seja mais detalhado nas explicações",
        "Sempre pergunte se o aluno entendeu",
        "Use emojis para tornar a explicação mais divertida"
    ],
    markdown=True
)

def teste_ab(agente_a, agente_b, perguntas_teste, nome_a="Agente A", nome_b="Agente B"):
    """Executa teste A/B entre dois agentes"""
    
    resultados_a = []
    resultados_b = []
    
    print(f"🥊 TESTE A/B: {nome_a} vs {nome_b}")
    print("=" * 50)
    
    for i, pergunta in enumerate(perguntas_teste, 1):
        print(f"\n🧪 Teste {i}: {pergunta}")
        
        # Testa Agente A
        print(f"  🤖 Testando {nome_a}...")
        metricas_a = coletar_metricas(agente_a, pergunta)
        metricas_a['agente'] = nome_a
        metricas_a['teste'] = i
        resultados_a.append(metricas_a)
        
        # Testa Agente B
        print(f"  🤖 Testando {nome_b}...")
        metricas_b = coletar_metricas(agente_b, pergunta)
        metricas_b['agente'] = nome_b
        metricas_b['teste'] = i
        resultados_b.append(metricas_b)
        
        # Comparação rápida
        if metricas_a['sucesso'] and metricas_b['sucesso']:
            if metricas_a['tempo_total'] < metricas_b['tempo_total']:
                print(f"    ⚡ {nome_a} foi mais rápido!")
            else:
                print(f"    ⚡ {nome_b} foi mais rápido!")
                
            if metricas_a['tokens_total'] < metricas_b['tokens_total']:
                print(f"    💰 {nome_a} usou menos tokens!")
            else:
                print(f"    💰 {nome_b} usou menos tokens!")
    
    return resultados_a + resultados_b

print("✅ Função de teste A/B criada!")

In [ ]:
# Executando teste A/B
perguntas_ab = [
    "Explique o que é uma fração",
    "Como resolver uma equação do segundo grau?",
    "O que é a função seno?"
]

resultados_ab = teste_ab(
    agente_teste, 
    agente_b, 
    perguntas_ab,
    "Agente Original", 
    "Agente Professor"
)

print("\n🎉 Teste A/B concluído!")

In [ ]:
# Análise dos resultados A/B
df_ab = pd.DataFrame(resultados_ab)

# Comparação por agente
comparacao = df_ab.groupby('agente').agg({
    'sucesso': 'mean',
    'tempo_total': 'mean',
    'tokens_total': 'mean'
}).round(3)

print("📊 RESULTADOS DO TESTE A/B")
print("=" * 40)
print(comparacao)

# Determina o vencedor
print("\n🏆 VENCEDORES POR CATEGORIA:")

# Taxa de sucesso
melhor_sucesso = comparacao['sucesso'].idxmax()
print(f"✅ Maior taxa de sucesso: {melhor_sucesso}")

# Menor tempo
mais_rapido = comparacao['tempo_total'].idxmin()
print(f"⚡ Mais rápido: {mais_rapido}")

# Menos tokens
mais_economico = comparacao['tokens_total'].idxmin()
print(f"💰 Mais econômico: {mais_economico}")

# Pontuação geral (métrica simples)
pontuacao = pd.DataFrame({
    'agente': comparacao.index,
    'pontos': 0
}).set_index('agente')

pontuacao.loc[melhor_sucesso, 'pontos'] += 1
pontuacao.loc[mais_rapido, 'pontos'] += 1
pontuacao.loc[mais_economico, 'pontos'] += 1

vencedor_geral = pontuacao['pontos'].idxmax()
print(f"\n🎯 VENCEDOR GERAL: {vencedor_geral}!")
print(f"Pontos: {pontuacao.loc[vencedor_geral, 'pontos']}/3")

## 🔍 Análise Avançada de Performance

Vamos fazer uma análise mais profunda! Como detetives da performance, vamos investigar todos os detalhes! 🕵️‍♂️

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_07.png)

In [ ]:
# Função para análise avançada de performance
def analisar_performance_avancada(resultados_df):
    """Análise detalhada de performance"""
    
    print("🔬 ANÁLISE AVANÇADA DE PERFORMANCE")
    print("=" * 50)
    
    # 1. Estatísticas descritivas
    print("\n📊 ESTATÍSTICAS DESCRITIVAS")
    colunas_numericas = ['tempo_total', 'tokens_total', 'tokens_entrada', 'tokens_saida']
    
    for coluna in colunas_numericas:
        if coluna in resultados_df.columns:
            dados = resultados_df[resultados_df['sucesso']][coluna]
            
            print(f"\n{coluna.replace('_', ' ').title()}:")
            print(f"  Média: {dados.mean():.3f}")
            print(f"  Mediana: {dados.median():.3f}")
            print(f"  Desvio Padrão: {dados.std():.3f}")
            print(f"  Min/Max: {dados.min():.3f} / {dados.max():.3f}")
            
            # Coeficiente de variação (estabilidade)
            cv = (dados.std() / dados.mean()) * 100 if dados.mean() > 0 else 0
            print(f"  Estabilidade (CV): {cv:.1f}%")
            
            if cv < 20:
                print(f"    ✅ Muito estável")
            elif cv < 50:
                print(f"    ⚠️ Moderadamente estável")
            else:
                print(f"    🚨 Instável")
    
    # 2. Análise de correlações
    print("\n🔗 ANÁLISE DE CORRELAÇÕES")
    df_sucesso = resultados_df[resultados_df['sucesso']]
    
    if len(df_sucesso) > 1:
        # Correlação tempo vs tokens
        corr_tempo_tokens = df_sucesso['tempo_total'].corr(df_sucesso['tokens_total'])
        print(f"Tempo vs Tokens: {corr_tempo_tokens:.3f}")
        
        if abs(corr_tempo_tokens) > 0.7:
            print("  📈 Forte correlação - tempo aumenta com tokens")
        elif abs(corr_tempo_tokens) > 0.3:
            print("  📊 Correlação moderada")
        else:
            print("  📉 Correlação fraca")
    
    # 3. Análise de outliers
    print("\n🎯 DETECÇÃO DE OUTLIERS")
    
    for coluna in ['tempo_total', 'tokens_total']:
        if coluna in df_sucesso.columns and len(df_sucesso) > 0:
            dados = df_sucesso[coluna]
            
            Q1 = dados.quantile(0.25)
            Q3 = dados.quantile(0.75)
            IQR = Q3 - Q1
            
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR
            
            outliers = dados[(dados < limite_inferior) | (dados > limite_superior)]
            
            print(f"\n{coluna.replace('_', ' ').title()}:")
            print(f"  Outliers encontrados: {len(outliers)}")
            
            if len(outliers) > 0:
                print(f"  Valores: {list(outliers.round(3))}")
                print(f"  🚨 Investigar casos extremos")
            else:
                print(f"  ✅ Sem outliers detectados")
    
    return {
        'total_testes': len(resultados_df),
        'taxa_sucesso': resultados_df['sucesso'].mean(),
        'estabilidade_tempo': (df_sucesso['tempo_total'].std() / df_sucesso['tempo_total'].mean()) if len(df_sucesso) > 0 else None
    }

# Aplicando análise aos nossos resultados
relatorio = analisar_performance_avancada(df_resultados)

print("\n📋 RELATÓRIO FINAL GERADO!")

## 🚨 Sistema de Alertas e SLA

Em produção, precisamos de alertas automáticos! Vamos criar um sistema que nos avisa quando algo não vai bem.

**SLA (Service Level Agreement)** = acordo de nível de serviço. É como uma "promessa" de qualidade!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_08.png)

In [ ]:
# Sistema de SLA e Alertas
class SistemaAlerta:
    def __init__(self):
        # Definindo SLAs (Service Level Agreement)
        self.slas = {
            'tempo_resposta_max': 5.0,  # segundos
            'taxa_sucesso_min': 95.0,   # porcentagem
            'tokens_max': 800,          # tokens por resposta
            'uptime_min': 99.0          # porcentagem de tempo online
        }
        
        self.historico_alertas = []
        self.metricas_periodo = []
        
    def verificar_slas(self, metricas_atuais):
        """Verifica se as métricas estão dentro do SLA"""
        violacoes = []
        
        # Verifica tempo de resposta
        if 'tempo_medio_recente' in metricas_atuais:
            if metricas_atuais['tempo_medio_recente'] > self.slas['tempo_resposta_max']:
                violacoes.append({
                    'tipo': 'TEMPO_RESPOSTA',
                    'valor_atual': metricas_atuais['tempo_medio_recente'],
                    'valor_sla': self.slas['tempo_resposta_max'],
                    'severidade': 'ALTA' if metricas_atuais['tempo_medio_recente'] > self.slas['tempo_resposta_max'] * 2 else 'MÉDIA'
                })
        
        # Verifica taxa de sucesso
        if 'taxa_sucesso_recente' in metricas_atuais:
            if metricas_atuais['taxa_sucesso_recente'] < self.slas['taxa_sucesso_min']:
                violacoes.append({
                    'tipo': 'TAXA_SUCESSO',
                    'valor_atual': metricas_atuais['taxa_sucesso_recente'],
                    'valor_sla': self.slas['taxa_sucesso_min'],
                    'severidade': 'CRÍTICA' if metricas_atuais['taxa_sucesso_recente'] < 80 else 'ALTA'
                })
        
        # Verifica consumo de tokens
        if 'tokens_medio_recente' in metricas_atuais:
            if metricas_atuais['tokens_medio_recente'] > self.slas['tokens_max']:
                violacoes.append({
                    'tipo': 'CONSUMO_TOKENS',
                    'valor_atual': metricas_atuais['tokens_medio_recente'],
                    'valor_sla': self.slas['tokens_max'],
                    'severidade': 'MÉDIA'
                })
        
        return violacoes
    
    def gerar_alerta(self, violacao):
        """Gera alerta baseado na violação"""
        timestamp = datetime.now()
        
        # Emojis por severidade
        emoji_severidade = {
            'CRÍTICA': '🚨',
            'ALTA': '⚠️',
            'MÉDIA': '⚡',
            'BAIXA': 'ℹ️'
        }
        
        # Mensagens por tipo
        mensagens = {
            'TEMPO_RESPOSTA': f"Tempo de resposta acima do SLA: {violacao['valor_atual']:.2f}s (limite: {violacao['valor_sla']}s)",
            'TAXA_SUCESSO': f"Taxa de sucesso abaixo do SLA: {violacao['valor_atual']:.1f}% (mínimo: {violacao['valor_sla']}%)",
            'CONSUMO_TOKENS': f"Consumo de tokens acima do SLA: {violacao['valor_atual']:.0f} (limite: {violacao['valor_sla']})"
        }
        
        alerta = {
            'timestamp': timestamp,
            'severidade': violacao['severidade'],
            'tipo': violacao['tipo'],
            'mensagem': mensagens.get(violacao['tipo'], 'Violação de SLA detectada'),
            'valor_atual': violacao['valor_atual'],
            'valor_sla': violacao['valor_sla']
        }
        
        self.historico_alertas.append(alerta)
        
        # Exibe alerta
        emoji = emoji_severidade.get(violacao['severidade'], '❓')
        print(f"{emoji} ALERTA {violacao['severidade']}: {alerta['mensagem']}")
        
        return alerta
    
    def relatorio_sla(self):
        """Gera relatório de cumprimento de SLA"""
        print("\n📊 RELATÓRIO DE SLA")
        print("=" * 30)
        
        if not self.historico_alertas:
            print("✅ Nenhuma violação de SLA detectada!")
            return
        
        # Conta violações por tipo
        violacoes_por_tipo = {}
        for alerta in self.historico_alertas:
            tipo = alerta['tipo']
            violacoes_por_tipo[tipo] = violacoes_por_tipo.get(tipo, 0) + 1
        
        print(f"Total de violações: {len(self.historico_alertas)}")
        for tipo, count in violacoes_por_tipo.items():
            print(f"  {tipo}: {count}")

# Criando sistema de alertas
sistema_alertas = SistemaAlerta()
print("🚨 Sistema de alertas configurado!")
print(f"SLAs definidos: {sistema_alertas.slas}")

In [ ]:
# Testando sistema de alertas
print("🧪 Testando sistema de alertas...")
print("=" * 40)

# Simulando métricas que violam SLA
metricas_teste = [
    {
        'tempo_medio_recente': 3.0,
        'taxa_sucesso_recente': 98.0,
        'tokens_medio_recente': 500,
        'descricao': 'Métricas normais'
    },
    {
        'tempo_medio_recente': 8.0,  # Viola SLA de tempo
        'taxa_sucesso_recente': 95.0,
        'tokens_medio_recente': 600,
        'descricao': 'Tempo alto'
    },
    {
        'tempo_medio_recente': 4.0,
        'taxa_sucesso_recente': 85.0,  # Viola SLA de sucesso
        'tokens_medio_recente': 700,
        'descricao': 'Taxa de sucesso baixa'
    },
    {
        'tempo_medio_recente': 6.0,   # Viola SLA de tempo
        'taxa_sucesso_recente': 90.0, # Viola SLA de sucesso
        'tokens_medio_recente': 900,  # Viola SLA de tokens
        'descricao': 'Múltiplas violações'
    }
]

for i, metricas in enumerate(metricas_teste, 1):
    print(f"\n🔍 Cenário {i}: {metricas['descricao']}")
    
    # Verifica SLAs
    violacoes = sistema_alertas.verificar_slas(metricas)
    
    if violacoes:
        print(f"  📊 {len(violacoes)} violação(ões) detectada(s):")
        for violacao in violacoes:
            sistema_alertas.gerar_alerta(violacao)
    else:
        print("  ✅ Todas as métricas dentro do SLA!")

# Relatório final
sistema_alertas.relatorio_sla()

## 📈 Otimização Baseada em Dados

Com todos esses dados coletados, agora podemos tomar decisões inteligentes para melhorar nossos agentes! 

**Dica!** A otimização não é "achismo" - é ciência! Use os dados para guiar suas decisões! 🧪

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_09.png)

In [ ]:
# Analisador de oportunidades de otimização
def analisar_oportunidades_otimizacao(df_resultados):
    """Identifica oportunidades de melhoria baseadas nos dados"""
    
    print("🎯 ANÁLISE DE OPORTUNIDADES DE OTIMIZAÇÃO")
    print("=" * 50)
    
    oportunidades = []
    
    # 1. Análise de falhas
    taxa_erro = (1 - df_resultados['sucesso'].mean()) * 100
    
    if taxa_erro > 10:
        oportunidades.append({
            'categoria': 'CONFIABILIDADE',
            'problema': f'Taxa de erro alta: {taxa_erro:.1f}%',
            'solucoes': [
                'Revisar instruções do agente',
                'Adicionar tratamento de erros',
                'Melhorar validação de entrada',
                'Implementar retry automático'
            ],
            'prioridade': 'ALTA'
        })
    
    # 2. Análise de performance
    df_sucesso = df_resultados[df_resultados['sucesso']]
    
    if len(df_sucesso) > 0:
        tempo_medio = df_sucesso['tempo_total'].mean()
        tempo_p95 = df_sucesso['tempo_total'].quantile(0.95)
        
        if tempo_medio > 5:
            oportunidades.append({
                'categoria': 'PERFORMANCE',
                'problema': f'Tempo médio alto: {tempo_medio:.2f}s',
                'solucoes': [
                    'Otimizar prompts para respostas mais diretas',
                    'Usar modelo mais rápido para tarefas simples',
                    'Implementar cache para respostas frequentes',
                    'Paralelizar chamadas quando possível'
                ],
                'prioridade': 'MÉDIA'
            })
        
        if tempo_p95 > tempo_medio * 2:
            oportunidades.append({
                'categoria': 'CONSISTÊNCIA',
                'problema': f'Alta variabilidade no tempo (P95: {tempo_p95:.2f}s vs média: {tempo_medio:.2f}s)',
                'solucoes': [
                    'Investigar casos que demoram muito',
                    'Implementar timeout para evitar travamentos',
                    'Padronizar complexidade das tarefas',
                    'Adicionar circuit breaker'
                ],
                'prioridade': 'MÉDIA'
            })
    
    # 3. Análise de custos
    if len(df_sucesso) > 0:
        tokens_medio = df_sucesso['tokens_total'].mean()
        tokens_max = df_sucesso['tokens_total'].max()
        
        if tokens_medio > 800:
            oportunidades.append({
                'categoria': 'CUSTO',
                'problema': f'Alto consumo de tokens: {tokens_medio:.0f} média',
                'solucoes': [
                    'Simplificar prompts desnecessariamente complexos',
                    'Usar modelos menores para tarefas simples',
                    'Implementar truncamento inteligente',
                    'Otimizar contexto para reduzir tokens de entrada'
                ],
                'prioridade': 'BAIXA'
            })
        
        if tokens_max > tokens_medio * 3:
            oportunidades.append({
                'categoria': 'CUSTO',
                'problema': f'Picos de consumo: {tokens_max} vs média {tokens_medio:.0f}',
                'soluções': [
                    'Identificar e otimizar casos que geram muitos tokens',
                    'Implementar limites de tokens por request',
                    'Usar streaming para interromper respostas longas'
                ],
                'prioridade': 'BAIXA'
            })
    
    # 4. Análise por categoria (se disponível)
    if 'categoria' in df_resultados.columns:
        pior_categoria = df_resultados.groupby('categoria')['sucesso'].mean().idxmin()
        taxa_pior = df_resultados.groupby('categoria')['sucesso'].mean().min() * 100
        
        if taxa_pior < 90:
            oportunidades.append({
                'categoria': 'ESPECIALIZAÇÃO',
                'problema': f'Baixa performance na categoria "{pior_categoria}": {taxa_pior:.1f}%',
                'solucoes': [
                    f'Adicionar exemplos específicos para {pior_categoria}',
                    f'Treinar agente especializado em {pior_categoria}',
                    f'Melhorar instruções para casos de {pior_categoria}',
                    'Implementar roteamento inteligente por categoria'
                ],
                'prioridade': 'MÉDIA'
            })
    
    # Exibe oportunidades
    if not oportunidades:
        print("🎉 Parabéns! Nenhuma oportunidade crítica de otimização encontrada!")
        return
    
    # Ordena por prioridade
    ordem_prioridade = {'ALTA': 1, 'MÉDIA': 2, 'BAIXA': 3}
    oportunidades.sort(key=lambda x: ordem_prioridade.get(x['prioridade'], 4))
    
    for i, oport in enumerate(oportunidades, 1):
        emoji_prioridade = {'ALTA': '🚨', 'MÉDIA': '⚠️', 'BAIXA': 'ℹ️'}
        emoji = emoji_prioridade.get(oport['prioridade'], '❓')
        
        print(f"\n{emoji} OPORTUNIDADE {i} - {oport['categoria']} (Prioridade {oport['prioridade']})")
        print(f"Problema: {oport['problema']}")
        print("Soluções sugeridas:")
        
        for j, solucao in enumerate(oport['solucoes'], 1):
            print(f"  {j}. {solucao}")
    
    return oportunidades

# Aplicando análise de otimização
oportunidades = analisar_oportunidades_otimizacao(df_resultados)

print("\n✅ Análise de otimização concluída!")

## 📊 Dashboard Final de Avaliação

Vamos criar um dashboard consolidado com todas as métricas importantes! É o nosso "painel de controle" definitivo! 🚀

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_10.png)

In [ ]:
# Dashboard consolidado
def criar_dashboard_completo(df_resultados, sistema_alertas):
    """Cria dashboard completo de avaliação"""
    
    # Configuração da figura
    fig, axes = plt.subplots(3, 3, figsize=(18, 15))
    fig.suptitle('🎯 DASHBOARD COMPLETO DE AVALIAÇÃO DE AGENTES', fontsize=20, fontweight='bold')
    
    # Remove eixos não utilizados
    axes = axes.flatten()
    
    # Dados para os gráficos
    df_sucesso = df_resultados[df_resultados['sucesso']]
    
    # 1. KPIs principais
    kpis = {
        'Taxa de Sucesso': f"{df_resultados['sucesso'].mean()*100:.1f}%",
        'Tempo Médio': f"{df_sucesso['tempo_total'].mean():.2f}s" if len(df_sucesso) > 0 else "N/A",
        'Tokens Médio': f"{df_sucesso['tokens_total'].mean():.0f}" if len(df_sucesso) > 0 else "N/A",
        'Total Testes': f"{len(df_resultados)}"
    }
    
    axes[0].axis('off')
    kpi_text = "\n".join([f"{k}: {v}" for k, v in kpis.items()])
    axes[0].text(0.5, 0.5, f"📊 KPIs PRINCIPAIS\n\n{kpi_text}", 
                ha='center', va='center', fontsize=14, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
    
    # 2. Taxa de sucesso por categoria
    if 'categoria' in df_resultados.columns:
        taxa_cat = df_resultados.groupby('categoria')['sucesso'].mean() * 100
        cores = plt.cm.Set3(np.linspace(0, 1, len(taxa_cat)))
        bars = axes[1].bar(taxa_cat.index, taxa_cat.values, color=cores, alpha=0.8)
        axes[1].set_title('Taxa de Sucesso por Categoria')
        axes[1].set_ylabel('Taxa (%)')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].grid(True, alpha=0.3)
        
        # Adiciona valores nas barras
        for bar, value in zip(bars, taxa_cat.values):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                        f'{value:.1f}%', ha='center', va='bottom')
    else:
        axes[1].text(0.5, 0.5, 'Dados de categoria\nnão disponíveis', 
                    ha='center', va='center', transform=axes[1].transAxes)
        axes[1].set_title('Taxa de Sucesso por Categoria')
    
    # 3. Distribuição de tempos
    if len(df_sucesso) > 0:
        axes[2].hist(df_sucesso['tempo_total'], bins=min(10, len(df_sucesso)), 
                    color='lightgreen', alpha=0.7, edgecolor='black')
        axes[2].axvline(df_sucesso['tempo_total'].mean(), color='red', linestyle='--', 
                       label=f'Média: {df_sucesso["tempo_total"].mean():.2f}s')
        axes[2].set_title('Distribuição de Tempos de Resposta')
        axes[2].set_xlabel('Tempo (s)')
        axes[2].set_ylabel('Frequência')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)
    else:
        axes[2].text(0.5, 0.5, 'Sem dados\nde sucesso', ha='center', va='center', 
                    transform=axes[2].transAxes)
        axes[2].set_title('Distribuição de Tempos')
    
    # 4. Correlação Tempo vs Tokens
    if len(df_sucesso) > 1:
        axes[3].scatter(df_sucesso['tokens_total'], df_sucesso['tempo_total'], 
                       alpha=0.6, color='purple')
        
        # Linha de tendência
        z = np.polyfit(df_sucesso['tokens_total'], df_sucesso['tempo_total'], 1)
        p = np.poly1d(z)
        axes[3].plot(df_sucesso['tokens_total'], p(df_sucesso['tokens_total']), 
                    "r--", alpha=0.8, label='Tendência')
        
        # Correlação
        corr = df_sucesso['tokens_total'].corr(df_sucesso['tempo_total'])
        
        axes[3].set_title(f'Tokens vs Tempo (Corr: {corr:.3f})')
        axes[3].set_xlabel('Tokens Total')
        axes[3].set_ylabel('Tempo (s)')
        axes[3].legend()
        axes[3].grid(True, alpha=0.3)
    else:
        axes[3].text(0.5, 0.5, 'Dados insuficientes\npara correlação', 
                    ha='center', va='center', transform=axes[3].transAxes)
        axes[3].set_title('Tokens vs Tempo')
    
    # 5. Timeline de performance
    if len(df_resultados) > 1:
        # Simula timeline (usa índice como proxy de tempo)
        timeline_sucesso = df_resultados['sucesso'].rolling(window=3, min_periods=1).mean() * 100
        axes[4].plot(range(len(timeline_sucesso)), timeline_sucesso, 
                    marker='o', linewidth=2, markersize=4, color='blue')
        axes[4].axhline(95, color='red', linestyle='--', alpha=0.7, label='Meta 95%')
        axes[4].set_title('Timeline de Taxa de Sucesso')
        axes[4].set_xlabel('Teste')
        axes[4].set_ylabel('Taxa Sucesso (%)')
        axes[4].legend()
        axes[4].grid(True, alpha=0.3)
    else:
        axes[4].text(0.5, 0.5, 'Dados insuficientes\npara timeline', 
                    ha='center', va='center', transform=axes[4].transAxes)
        axes[4].set_title('Timeline de Performance')
    
    # 6. Status dos SLAs
    sla_status = {
        'Tempo': '✅' if df_sucesso['tempo_total'].mean() <= 5.0 else '❌',
        'Sucesso': '✅' if df_resultados['sucesso'].mean() >= 0.95 else '❌',
        'Tokens': '✅' if df_sucesso['tokens_total'].mean() <= 800 else '❌'
    } if len(df_sucesso) > 0 else {'Status': 'N/A'}
    
    axes[5].axis('off')
    sla_text = "\n".join([f"{k}: {v}" for k, v in sla_status.items()])
    axes[5].text(0.5, 0.5, f"📋 STATUS SLA\n\n{sla_text}", 
                ha='center', va='center', fontsize=14,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8))
    
    # 7. Distribuição de tokens
    if len(df_sucesso) > 0:
        # Boxplot de tokens
        axes[6].boxplot([df_sucesso['tokens_total']], labels=['Tokens'])
        axes[6].set_title('Distribuição de Tokens')
        axes[6].set_ylabel('Tokens')
        axes[6].grid(True, alpha=0.3)
        
        # Estatísticas
        q1, q3 = df_sucesso['tokens_total'].quantile([0.25, 0.75])
        median = df_sucesso['tokens_total'].median()
        axes[6].text(1.2, median, f'Mediana: {median:.0f}', va='center')
    else:
        axes[6].text(0.5, 0.5, 'Sem dados\nde tokens', ha='center', va='center', 
                    transform=axes[6].transAxes)
        axes[6].set_title('Distribuição de Tokens')
    
    # 8. Alertas recentes
    alertas_recentes = sistema_alertas.historico_alertas[-5:] if sistema_alertas.historico_alertas else []
    
    axes[7].axis('off')
    if alertas_recentes:
        alertas_text = "\n".join([f"• {a['tipo']}: {a['severidade']}" for a in alertas_recentes])
        axes[7].text(0.5, 0.5, f"🚨 ALERTAS RECENTES\n\n{alertas_text}", 
                    ha='center', va='center', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.8))
    else:
        axes[7].text(0.5, 0.5, "🟢 SISTEMA SAUDÁVEL\n\nNenhum alerta ativo", 
                    ha='center', va='center', fontsize=12,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
    
    # 9. Resumo de recomendações
    axes[8].axis('off')
    
    # Calcula score geral (0-100)
    score_sucesso = df_resultados['sucesso'].mean() * 40  # 40 pontos máximo
    score_tempo = min(40, (5.0 / df_sucesso['tempo_total'].mean()) * 40) if len(df_sucesso) > 0 and df_sucesso['tempo_total'].mean() > 0 else 0
    score_tokens = min(20, (800 / df_sucesso['tokens_total'].mean()) * 20) if len(df_sucesso) > 0 and df_sucesso['tokens_total'].mean() > 0 else 0
    
    score_final = score_sucesso + score_tempo + score_tokens
    
    if score_final >= 90:
        status_geral = "🌟 EXCELENTE"
        cor_status = "lightgreen"
    elif score_final >= 70:
        status_geral = "👍 BOM"
        cor_status = "lightyellow"
    elif score_final >= 50:
        status_geral = "⚠️ REGULAR"
        cor_status = "orange"
    else:
        status_geral = "🚨 CRÍTICO"
        cor_status = "lightcoral"
    
    axes[8].text(0.5, 0.5, f"📊 SCORE GERAL\n\n{score_final:.1f}/100\n\n{status_geral}", 
                ha='center', va='center', fontsize=14, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor=cor_status, alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Relatório textual
    print(f"\n🎯 SCORE FINAL: {score_final:.1f}/100 - {status_geral}")
    print(f"\n📊 Breakdown do Score:")
    print(f"  • Sucesso (40pts): {score_sucesso:.1f}")
    print(f"  • Performance (40pts): {score_tempo:.1f}")
    print(f"  • Eficiência (20pts): {score_tokens:.1f}")
    
    return {
        'score_final': score_final,
        'status': status_geral,
        'breakdown': {
            'sucesso': score_sucesso,
            'performance': score_tempo,
            'eficiencia': score_tokens
        }
    }

# Criando dashboard
print("🚀 Gerando dashboard completo...")
score_resultado = criar_dashboard_completo(df_resultados, sistema_alertas)

print("\n✅ Dashboard gerado com sucesso! 🎨")

## 🎓 Exercícios Práticos

Agora é sua vez de praticar! Vamos fazer alguns exercícios para fixar o conhecimento! 💪

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_11.png)

In [ ]:
# EXERCÍCIO 1: Crie um novo agente e avalie sua performance
print("🎯 EXERCÍCIO 1: Criando e Avaliando Novo Agente")
print("=" * 50)
print("\n📋 INSTRUÇÕES:")
print("1. Crie um agente especializado em OUTRO domínio (não matemática)")
print("2. Defina pelo menos 5 casos de teste")
print("3. Execute os testes e colete métricas")
print("4. Analise os resultados")
print("\n💡 Sugestões de domínios: culinária, programação, história, ciências...")

# ESPAÇO PARA SEU CÓDIGO:
# Crie seu agente aqui
# agente_exercicio = Agent(...)

print("\n✏️ Complete o código acima com sua solução!")

In [ ]:
# EXERCÍCIO 2: Implementar métrica customizada
print("🎯 EXERCÍCIO 2: Métrica de Qualidade Customizada")
print("=" * 50)
print("\n📋 INSTRUÇÕES:")
print("1. Crie uma função que avalie a QUALIDADE da resposta")
print("2. Use critérios como: clareza, completude, precisão")
print("3. Retorne uma nota de 0 a 10")
print("4. Teste com algumas respostas do agente")

def avaliar_qualidade_resposta(pergunta, resposta):
    """Avalia a qualidade de uma resposta (0-10)"""
    score = 0
    
    # COMPLETE AQUI: Implemente sua lógica de avaliação
    # Exemplo de critérios:
    # - Resposta não é vazia (+2 pontos)
    # - Resposta tem tamanho adequado (+2 pontos)
    # - Resposta contém palavras-chave da pergunta (+2 pontos)
    # - Resposta é estruturada (parágrafos, listas) (+2 pontos)
    # - Resposta é educativa/explicativa (+2 pontos)
    
    return min(score, 10)  # Máximo 10 pontos

# Teste sua função aqui
exemplo_pergunta = "Explique o que é inteligência artificial"
exemplo_resposta = "IA é a capacidade de máquinas simularem inteligência humana..."

nota = avaliar_qualidade_resposta(exemplo_pergunta, exemplo_resposta)
print(f"\n🎯 Nota da resposta: {nota}/10")

print("\n✏️ Complete a função acima para avaliar qualidade!")

In [ ]:
# EXERCÍCIO 3: Benchmark entre modelos
print("🎯 EXERCÍCIO 3: Benchmark de Modelos")
print("=" * 50)
print("\n📋 INSTRUÇÕES:")
print("1. Compare diferentes modelos do Gemini (se disponível)")
print("2. Use as mesmas perguntas para todos")
print("3. Compare tempo, tokens e qualidade")
print("4. Determine qual é melhor para cada tipo de tarefa")

# Exemplo de estrutura:
modelos_teste = [
    # COMPLETE: Adicione diferentes modelos para comparar
    # {"nome": "Gemini Flash", "id": "gemini-2.0-flash-exp"},
    # {"nome": "Gemini Pro", "id": "gemini-pro"},
]

perguntas_benchmark = [
    "Explique brevemente o que é machine learning",
    "Resolva: 2x + 5 = 15",
    "Conte uma história curta sobre robôs"
]

def executar_benchmark(modelos, perguntas):
    """Executa benchmark entre diferentes modelos"""
    resultados = []
    
    # COMPLETE AQUI: Implemente o benchmark
    # Para cada modelo:
    #   Para cada pergunta:
    #     Execute e colete métricas
    #     Armazene resultados
    
    return resultados

print("\n✏️ Complete o código para fazer benchmark entre modelos!")

## 📚 Resumo e Próximos Passos

Uau! Que jornada incrível! Vamos recapitular tudo que aprendemos sobre avaliação de agentes! 🎉

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-11_img_12.png)

### 🎯 O que aprendemos:

#### 📊 **Métricas Fundamentais**
- **Performance**: Tempo de resposta, throughput
- **Qualidade**: Precisão, relevância, coerência
- **Confiabilidade**: Taxa de sucesso, estabilidade
- **Eficiência**: Consumo de tokens, custos

#### 🧪 **Metodologias de Teste**
- Baterias de teste automatizadas
- Testes A/B entre agentes
- Análise de correlações
- Detecção de outliers

#### 🚨 **Monitoramento e Alertas**
- SLAs (Service Level Agreements)
- Alertas automáticos por severidade
- Monitoramento em tempo real
- Dashboards executivos

#### 🎯 **Otimização Baseada em Dados**
- Identificação de gargalos
- Oportunidades de melhoria
- Priorização de problemas
- ROI de otimizações

In [ ]:
# Checklist final - verifique se você domina esses conceitos!
checklist_avaliacao = {
    "📊 Coleta de métricas básicas": "✅",
    "🧪 Execução de testes automatizados": "✅",
    "📈 Criação de dashboards": "✅",
    "🚨 Configuração de alertas": "✅",
    "🔍 Análise de performance": "✅",
    "⚡ Identificação de otimizações": "✅",
    "🏆 Testes A/B entre agentes": "✅",
    "📋 Relatórios executivos": "✅"
}

print("🎓 CHECKLIST DE COMPETÊNCIAS - MÓDULO 11")
print("=" * 50)

for competencia, status in checklist_avaliacao.items():
    print(f"{status} {competencia}")

print("\n🎉 PARABÉNS! Você agora é um especialista em avaliação de agentes!")
print("\n🚀 PRÓXIMO MÓDULO: AgentOS em Produção")
print("Vamos aprender a colocar nossos agentes avaliados em produção!")

### 🔥 **Dicas Finais do Pedro:**

1. **Sempre meça antes de otimizar** - "O que não se mede, não se gerencia!"
2. **Defina SLAs realistas** - Nem muito permissivos, nem impossíveis
3. **Automatize tudo que puder** - Testes manuais não escalam
4. **Monitore em produção** - O comportamento real pode surpreender
5. **Use dados para justificar mudanças** - Stakeholders adoram gráficos! 📊

### 🎯 **Para o Próximo Módulo:**
- Vamos ver como usar o **AgentOS** para produção
- Deploy e monitoramento em ambiente real
- Escalabilidade e alta disponibilidade
- Integração com sistemas empresariais

**Continuem praticando e até o próximo módulo! Vocês estão arrasando! 🚀**

---
*Pedro Nunes Guth - Especialista em IA e AWS*